## Imports

In [1]:
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
#Keras imports
import tensorflow.keras.preprocessing as preprocessing
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import models
from keras import optimizers

Using TensorFlow backend.


## Pathways

In [26]:
not_path = os.listdir("../Data/Images")
not_path.remove(".DS_Store") #if it's still in there, it likes to pop up from time to time
path_to_data = "../Data"

test_path = os.path.join(data_path, 'test_images')
train_path = os.path.join(data_path, 'train_images')
val_path = os.path.join(data_path, 'val_images')

data_path = os.path.join('..', 'Data')
images_path = os.path.join(data_path, 'Images')
image_folders = os.listdir(images_path)
test_folders = os.listdir(test_path)
train_folders = os.listdir(train_path)
val_folders = os.listdir(val_path)

In [27]:
# using pugs, goldens, and pembrokes (AKA corgis)
train_dir = "../Data/ExModeling_train"
test_dir = "../Data/ExModeling_test"
val_dir = "../Data/ExModeling_val"

## First Simple Model

In [ ]:
#emphasis on simple, not even using validation data
fsm_train_dir = "../Data/FSMimgs"#only contains 'bluetick' and 'boxer' breeds
fsm_train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        fsm_train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical')
fsm_train_generator.class_indices

In [ ]:
fsm_model = models.Sequential()
fsm_model.add(layers.Dense(64, input_shape = (150, 150, 3)))
fsm_model.add(layers.Flatten())
fsm_model.add(layers.Dense(2, activation = 'softmax'))
fsm_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [ ]:
history = fsm_model.fit_generator(train_generator, steps_per_epoch=100, epochs=5)
#neted a training accuracy of 0.4688, somehow less than half with two classes

## Set up

In [28]:
# easy param tuning
img_size = 128
bch_size = 20

In [50]:
img_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, vertical_flip=True)

train_generator = img_gen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    color_mode='grayscale',
    batch_size=bch_size,
    class_mode='categorical')

validation_generator = img_gen.flow_from_directory(
    val_dir, 
    target_size=(img_size, img_size), 
    color_mode='grayscale',
    batch_size=bch_size, 
    class_mode='categorical')

test_generator = img_gen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    color_mode='grayscale',
    batch_size=bch_size,
    class_mode='categorical',
    shuffle=False)


Found 664 images belonging to 5 classes.
Found 145 images belonging to 5 classes.
Found 141 images belonging to 5 classes.


## Modeling

In [12]:
#refrenced material: https://github.com/learn-co-students/dsc-image-classification-lab-sea01-dtsc-ft-051120/tree/solution
#even has the same dataset, that's what I get for using the first google result of 'kaggle dog image dataset'
#model 1: 3 dogs, Dense and Convid2D, img_size = 128
model_1 = models.Sequential()
model_1.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))#imput layer
model_1.add(layers.MaxPooling2D((2, 2)))
model_1.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_1.add(layers.MaxPooling2D((2, 2)))
model_1.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_1.add(layers.MaxPooling2D((2, 2)))
model_1.add(layers.Flatten())
model_1.add(layers.Dense(64, activation='relu'))
model_1.add(layers.Dense(128, activation='relu'))
model_1.add(layers.Dense(3, activation='softmax'))#output layer w/ softmax

model_1.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

W0810 14:43:03.797093 4631838144 deprecation_wrapper.py:119] From /Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0810 14:43:03.858423 4631838144 deprecation_wrapper.py:119] From /Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0810 14:43:03.867154 4631838144 deprecation_wrapper.py:119] From /Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0810 14:43:03.901729 4631838144 deprecation_wrapper.py:119] From /Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.m

In [13]:
history_1 = model_1.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50)

W0810 14:44:43.136464 4631838144 deprecation.py:323] From /Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0810 14:44:43.294936 4631838144 deprecation_wrapper.py:119] From /Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/10
100/100 [==============================] - 46s 463ms/step - loss: 1.0545 - acc: 0.4389 - val_loss: 1.0658 - val_acc: 0.4286
Epoch 2/10
100/100 [==============================] - 60s 601ms/step - loss: 0.8852 - acc: 0.6138 - val_loss: 0.9146 - val_acc: 0.5714
Epoch 3/10
100/100 [==============================] - 45s 450ms/step - loss: 0.7634 - acc: 0.6784 - val_loss: 0.9709 - val_acc: 0.4286
Epoch 4/10
100/100 [==============================] - 46s 458ms/step - loss: 0.6736 - acc: 0.7257 - val_loss: 1.0234 - val_acc: 0.4286
Epoch 5/10
100/100 [==============================] - 56s 564ms/step - loss: 0.6076 - acc: 0.7526 - val_loss: 0.9976 - val_acc: 0.4286
Epoch 6/10
100/100 [==============================] - 49s 493ms/step - loss: 0.5093 - acc: 0.8094 - val_loss: 1.0799 - val_acc: 0.5714
Epoch 7/10
100/100 [==============================] - 54s 538ms/step - loss: 0.4053 - acc: 0.8505 - val_loss: 1.1888 - val_acc: 0.5714
Epoch 8/10
100/100 [==============================] - 4

In [ ]:
#added pomeranians and saluki for more classes, added more neurons

In [20]:
model_2 = models.Sequential()
model_2.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))#imput layer
model_2.add(layers.MaxPooling2D((2, 2)))
model_2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_2.add(layers.MaxPooling2D((2, 2)))
model_2.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_2.add(layers.MaxPooling2D((2, 2)))
model_2.add(layers.Conv2D(256, (3, 3), activation='relu'))
model_2.add(layers.MaxPooling2D((2, 2)))
model_2.add(layers.Flatten())
model_2.add(layers.Dense(64, activation='relu'))
model_2.add(layers.Dense(128, activation='relu'))
model_2.add(layers.Dense(256, activation='relu'))
model_2.add(layers.Dense(5, activation='softmax'))#output layer w/ softmax, changed nodes to 5, =n

model_2.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

In [21]:
history_2 = model_2.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50)

Epoch 1/10
100/100 [==============================] - 71s 707ms/step - loss: 1.5879 - acc: 0.2458 - val_loss: 1.6248 - val_acc: 0.2500
Epoch 2/10
100/100 [==============================] - 53s 527ms/step - loss: 1.4926 - acc: 0.3547 - val_loss: 1.5292 - val_acc: 0.5833
Epoch 3/10
100/100 [==============================] - 56s 559ms/step - loss: 1.4138 - acc: 0.3906 - val_loss: 1.4466 - val_acc: 0.4167
Epoch 4/10
100/100 [==============================] - 57s 568ms/step - loss: 1.3638 - acc: 0.4211 - val_loss: 1.3588 - val_acc: 0.4167
Epoch 5/10
100/100 [==============================] - 50s 503ms/step - loss: 1.3065 - acc: 0.4616 - val_loss: 1.4726 - val_acc: 0.4167
Epoch 6/10
100/100 [==============================] - 59s 589ms/step - loss: 1.2456 - acc: 0.5034 - val_loss: 1.3056 - val_acc: 0.5833
Epoch 7/10
100/100 [==============================] - 51s 509ms/step - loss: 1.2044 - acc: 0.5152 - val_loss: 1.4435 - val_acc: 0.5000
Epoch 8/10
100/100 [==============================] - 5

In [22]:
history_3 = model_2.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=17, #acc was improving each epoch, needs more, trying between 15 and 20
    validation_data=validation_generator,
    validation_steps=50)

Epoch 1/17
100/100 [==============================] - 56s 559ms/step - loss: 0.9191 - acc: 0.6364 - val_loss: 1.5649 - val_acc: 0.4167
Epoch 2/17
100/100 [==============================] - 57s 566ms/step - loss: 0.8247 - acc: 0.6919 - val_loss: 1.5214 - val_acc: 0.5000
Epoch 3/17
100/100 [==============================] - 52s 523ms/step - loss: 0.7221 - acc: 0.7389 - val_loss: 1.4327 - val_acc: 0.3333
Epoch 4/17
100/100 [==============================] - 59s 588ms/step - loss: 0.6411 - acc: 0.7741 - val_loss: 1.9297 - val_acc: 0.4167
Epoch 5/17
100/100 [==============================] - 51s 510ms/step - loss: 0.5018 - acc: 0.8292 - val_loss: 1.6266 - val_acc: 0.4167
Epoch 6/17
100/100 [==============================] - 52s 515ms/step - loss: 0.3874 - acc: 0.8622 - val_loss: 1.7267 - val_acc: 0.4167
Epoch 7/17
100/100 [==============================] - 59s 590ms/step - loss: 0.3269 - acc: 0.8879 - val_loss: 2.0073 - val_acc: 0.2500
Epoch 8/17
100/100 [==============================] - 5

In [23]:
#oops, looks like it ran 27 times, val acc is low, most val_image folders of breeds have 2-3 images, more might be needed

In [24]:
#new plan is to run a 5 breed model with train 70% test 15% val 15% rather than the current 90 9 1 
#Back to the cleaning_n_prep notebook

In [ ]:
#70 15 15 achieved, lets try the same breeds in hopes of better val acc with model 3


In [30]:
model_3 = models.Sequential()
model_3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))#imput layer
model_3.add(layers.MaxPooling2D((2, 2)))
model_3.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_3.add(layers.MaxPooling2D((2, 2)))
model_3.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_3.add(layers.MaxPooling2D((2, 2)))
model_3.add(layers.Conv2D(256, (3, 3), activation='relu'))
model_3.add(layers.MaxPooling2D((2, 2)))
model_3.add(layers.Flatten())
model_3.add(layers.Dense(64, activation='relu'))
model_3.add(layers.Dense(128, activation='relu'))
model_3.add(layers.Dense(256, activation='relu'))
model_3.add(layers.Dense(5, activation='softmax'))

model_3.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

In [31]:
history_4 = model_3.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=17, #still trying between 15 and 20 this time
    validation_data=validation_generator,
    validation_steps=50)

Epoch 1/17
100/100 [==============================] - 56s 560ms/step - loss: 1.5967 - acc: 0.2590 - val_loss: 1.5728 - val_acc: 0.2604
Epoch 2/17
100/100 [==============================] - 63s 632ms/step - loss: 1.5123 - acc: 0.3330 - val_loss: 1.4747 - val_acc: 0.3505
Epoch 3/17
100/100 [==============================] - 54s 541ms/step - loss: 1.4312 - acc: 0.3666 - val_loss: 1.5666 - val_acc: 0.2945
Epoch 4/17
100/100 [==============================] - 52s 521ms/step - loss: 1.3662 - acc: 0.4055 - val_loss: 1.4500 - val_acc: 0.3609
Epoch 5/17
100/100 [==============================] - 61s 608ms/step - loss: 1.3383 - acc: 0.4310 - val_loss: 1.3917 - val_acc: 0.4110
Epoch 6/17
100/100 [==============================] - 53s 529ms/step - loss: 1.2859 - acc: 0.4705 - val_loss: 1.3970 - val_acc: 0.3956
Epoch 7/17
100/100 [==============================] - 59s 593ms/step - loss: 1.2298 - acc: 0.4944 - val_loss: 1.4753 - val_acc: 0.3802
Epoch 8/17
100/100 [==============================] - 5

In [32]:
test_loss, test_acc = model_3.evaluate_generator(test_generator, steps=50)
print(test_loss)
print(test_acc)


2.5339999952380867
0.4345372494133547


In [33]:
#I'm overfitting, is the model is too complex?

In [34]:
model_4 = models.Sequential()
model_4.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))#imput layer
model_4.add(layers.MaxPooling2D((2, 2)))
model_4.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_4.add(layers.MaxPooling2D((2, 2)))
model_4.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_4.add(layers.MaxPooling2D((2, 2)))
#model_3.add(layers.Conv2D(256, (3, 3), activation='relu'))
#model_3.add(layers.MaxPooling2D((2, 2)))
model_4.add(layers.Flatten())
model_4.add(layers.Dense(64, activation='relu'))
model_4.add(layers.Dense(128, activation='relu'))
#model_3.add(layers.Dense(256, activation='relu'))
model_4.add(layers.Dense(5, activation='softmax'))

model_4.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])
#cut some layers in an attempt to fight overfitting

In [35]:
history_5 = model_4.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=10, #10 for now
    validation_data=validation_generator,
    validation_steps=50)

Epoch 1/10
100/100 [==============================] - 69s 694ms/step - loss: 1.5895 - acc: 0.2670 - val_loss: 1.5444 - val_acc: 0.3637
Epoch 2/10
100/100 [==============================] - 75s 751ms/step - loss: 1.4508 - acc: 0.3890 - val_loss: 1.3995 - val_acc: 0.4846
Epoch 3/10
100/100 [==============================] - 69s 694ms/step - loss: 1.2930 - acc: 0.4825 - val_loss: 1.4899 - val_acc: 0.3721
Epoch 4/10
100/100 [==============================] - 56s 560ms/step - loss: 1.1475 - acc: 0.5540 - val_loss: 1.3800 - val_acc: 0.4275
Epoch 5/10
100/100 [==============================] - 47s 466ms/step - loss: 0.9740 - acc: 0.6535 - val_loss: 1.4785 - val_acc: 0.4011
Epoch 6/10
100/100 [==============================] - 47s 471ms/step - loss: 0.8187 - acc: 0.7310 - val_loss: 1.5285 - val_acc: 0.3923
Epoch 7/10
100/100 [==============================] - 56s 565ms/step - loss: 0.6316 - acc: 0.7960 - val_loss: 2.0100 - val_acc: 0.3799
Epoch 8/10
100/100 [==============================] - 5

In [36]:
#Fine, I'll dumb it down even more
model_5 = models.Sequential()
model_5.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))#imput layer
model_5.add(layers.MaxPooling2D((2, 2)))
model_5.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_5.add(layers.MaxPooling2D((2, 2)))
#model_4.add(layers.Conv2D(128, (3, 3), activation='relu'))
#model_4.add(layers.MaxPooling2D((2, 2)))
#model_3.add(layers.Conv2D(256, (3, 3), activation='relu'))
#model_3.add(layers.MaxPooling2D((2, 2)))
model_5.add(layers.Flatten())
model_5.add(layers.Dense(64, activation='relu'))
#model_4.add(layers.Dense(128, activation='relu'))
#model_3.add(layers.Dense(256, activation='relu'))
model_5.add(layers.Dense(5, activation='softmax'))

model_5.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

In [37]:
history_6 = model_5.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=10, 
    validation_data=validation_generator,
    validation_steps=50)

Epoch 1/10
100/100 [==============================] - 40s 395ms/step - loss: 1.6081 - acc: 0.2605 - val_loss: 1.6790 - val_acc: 0.1791
Epoch 2/10
100/100 [==============================] - 47s 470ms/step - loss: 1.4357 - acc: 0.4250 - val_loss: 1.4522 - val_acc: 0.4220
Epoch 3/10
100/100 [==============================] - 38s 378ms/step - loss: 1.2467 - acc: 0.5326 - val_loss: 1.3751 - val_acc: 0.4253
Epoch 4/10
100/100 [==============================] - 41s 406ms/step - loss: 1.0825 - acc: 0.6225 - val_loss: 1.4232 - val_acc: 0.3866
Epoch 5/10
100/100 [==============================] - 49s 490ms/step - loss: 0.9227 - acc: 0.6845 - val_loss: 1.3858 - val_acc: 0.4088
Epoch 6/10
100/100 [==============================] - 38s 375ms/step - loss: 0.7724 - acc: 0.7675 - val_loss: 1.5718 - val_acc: 0.3901
Epoch 7/10
100/100 [==============================] - 37s 375ms/step - loss: 0.6457 - acc: 0.8120 - val_loss: 1.6193 - val_acc: 0.3330
Epoch 8/10
100/100 [==============================] - 3

In [41]:
test_loss, test_acc = model_5.evaluate_generator(test_generator, steps=50)
print(test_loss)
print(test_acc)


1.9381213243335957
0.3261851013951323


In [42]:
#added flips now to the data gen, let's see if it helps
model_6 = models.Sequential()
model_6.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))#imput layer
model_6.add(layers.MaxPooling2D((2, 2)))
model_6.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_6.add(layers.MaxPooling2D((2, 2)))
#model_4.add(layers.Conv2D(128, (3, 3), activation='relu'))
#model_4.add(layers.MaxPooling2D((2, 2)))
#model_3.add(layers.Conv2D(256, (3, 3), activation='relu'))
#model_3.add(layers.MaxPooling2D((2, 2)))
model_6.add(layers.Flatten())
model_6.add(layers.Dense(32, activation='relu'))#changed to 32 to further simplify
#model_4.add(layers.Dense(128, activation='relu'))
#model_3.add(layers.Dense(256, activation='relu'))
model_6.add(layers.Dense(5, activation='softmax'))

model_6.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

In [43]:
history_7 = model_6.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=10, 
    validation_data=validation_generator,
    validation_steps=50)

Epoch 1/10
100/100 [==============================] - 42s 421ms/step - loss: 1.5937 - acc: 0.2655 - val_loss: 1.5568 - val_acc: 0.3396
Epoch 2/10
100/100 [==============================] - 40s 400ms/step - loss: 1.5162 - acc: 0.3360 - val_loss: 1.5545 - val_acc: 0.3308
Epoch 3/10
100/100 [==============================] - 41s 414ms/step - loss: 1.4701 - acc: 0.3725 - val_loss: 1.4976 - val_acc: 0.4033
Epoch 4/10
100/100 [==============================] - 41s 410ms/step - loss: 1.4395 - acc: 0.3865 - val_loss: 1.4972 - val_acc: 0.3698
Epoch 5/10
100/100 [==============================] - 59s 590ms/step - loss: 1.4204 - acc: 0.3980 - val_loss: 1.5011 - val_acc: 0.3890
Epoch 6/10
100/100 [==============================] - 72s 720ms/step - loss: 1.3845 - acc: 0.4185 - val_loss: 1.5664 - val_acc: 0.3978
Epoch 7/10
100/100 [==============================] - 43s 434ms/step - loss: 1.3886 - acc: 0.4120 - val_loss: 1.4869 - val_acc: 0.3912
Epoch 8/10
100/100 [==============================] - 4

In [44]:
test_loss, test_acc = model_6.evaluate_generator(test_generator, steps=50)
print(test_loss)
print(test_acc)

1.474687071470859
0.3679458259459812


In [45]:
#added flips now to the data gen, let's see if it helps
model_7 = models.Sequential()
model_7.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))#imput layer
model_7.add(layers.MaxPooling2D((2, 2)))
model_7.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_7.add(layers.MaxPooling2D((2, 2)))
model_7.add(layers.Conv2D(128, (3, 3), activation='relu'))#brough another Convolution layer back
model_7.add(layers.MaxPooling2D((2, 2)))
#model_3.add(layers.Conv2D(256, (3, 3), activation='relu'))
#model_3.add(layers.MaxPooling2D((2, 2)))
model_7.add(layers.Flatten())
model_7.add(layers.Dense(64, activation='relu'))#changed to back to 64 to add complexity
#model_4.add(layers.Dense(128, activation='relu'))
#model_3.add(layers.Dense(256, activation='relu'))
model_7.add(layers.Dense(5, activation='softmax'))

model_7.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

In [46]:
history_8 = model_7.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=12, #just a little more than last time
    validation_data=validation_generator,
    validation_steps=50)

Epoch 1/12
100/100 [==============================] - 71s 711ms/step - loss: 1.5441 - acc: 0.2970 - val_loss: 1.4327 - val_acc: 0.4220
Epoch 2/12
100/100 [==============================] - 49s 487ms/step - loss: 1.4300 - acc: 0.3855 - val_loss: 1.4183 - val_acc: 0.4145
Epoch 3/12
100/100 [==============================] - 48s 483ms/step - loss: 1.3599 - acc: 0.4245 - val_loss: 1.3667 - val_acc: 0.4143
Epoch 4/12
100/100 [==============================] - 56s 564ms/step - loss: 1.3216 - acc: 0.4470 - val_loss: 1.4310 - val_acc: 0.3725
Epoch 5/12
100/100 [==============================] - 48s 477ms/step - loss: 1.2648 - acc: 0.4935 - val_loss: 1.3885 - val_acc: 0.4132
Epoch 6/12
100/100 [==============================] - 47s 471ms/step - loss: 1.2265 - acc: 0.5161 - val_loss: 1.3714 - val_acc: 0.4179
Epoch 7/12
100/100 [==============================] - 55s 547ms/step - loss: 1.1906 - acc: 0.5335 - val_loss: 1.3785 - val_acc: 0.4066
Epoch 8/12
100/100 [==============================] - 4

In [47]:
test_loss, test_acc = model_7.evaluate_generator(test_generator, steps=50)
print(test_loss)
print(test_acc)

1.4178522961362423
0.4521337918069261


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 28, 28, 128)       73856     
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 14, 14, 128)       0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 25088)             0         
__________

In [53]:
#gray-scaled all images imput shape changed to 1 as no longer rgb
model_8 = models.Sequential()
model_8.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 1)))
model_8.add(layers.MaxPooling2D((2, 2)))
model_8.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_8.add(layers.MaxPooling2D((2, 2)))
model_8.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_8.add(layers.MaxPooling2D((2, 2)))
#model_3.add(layers.Conv2D(256, (3, 3), activation='relu'))
#model_3.add(layers.MaxPooling2D((2, 2)))
model_8.add(layers.Flatten())
model_8.add(layers.Dense(64, activation='relu'))
model_8.add(layers.Dense(128, activation='relu'))#brough anohter dense layer back
#model_3.add(layers.Dense(256, activation='relu'))
model_8.add(layers.Dense(5, activation='softmax'))

model_8.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

In [54]:
history_9 = model_8.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=10, 
    validation_data=validation_generator,
    validation_steps=50)

Epoch 1/10
100/100 [==============================] - 50s 498ms/step - loss: 1.5996 - acc: 0.2440 - val_loss: 1.5954 - val_acc: 0.2165
Epoch 2/10
100/100 [==============================] - 48s 476ms/step - loss: 1.5413 - acc: 0.3190 - val_loss: 1.5303 - val_acc: 0.3648
Epoch 3/10
100/100 [==============================] - 56s 563ms/step - loss: 1.4851 - acc: 0.3595 - val_loss: 1.5316 - val_acc: 0.3564
Epoch 4/10
100/100 [==============================] - 47s 471ms/step - loss: 1.4615 - acc: 0.3660 - val_loss: 1.5455 - val_acc: 0.3714
Epoch 5/10
100/100 [==============================] - 46s 461ms/step - loss: 1.4479 - acc: 0.3795 - val_loss: 1.5470 - val_acc: 0.2934
Epoch 6/10
100/100 [==============================] - 57s 571ms/step - loss: 1.4358 - acc: 0.3880 - val_loss: 1.5278 - val_acc: 0.3615
Epoch 7/10
100/100 [==============================] - 47s 468ms/step - loss: 1.4121 - acc: 0.3995 - val_loss: 1.5121 - val_acc: 0.3084
Epoch 8/10
100/100 [==============================] - 5

In [55]:
test_loss, test_acc = model_8.evaluate_generator(test_generator, steps=50)
print(test_loss)
print(test_acc)#we're underfitting again, time to up the complexity

1.4444455312014164
0.4232505655786523


In [57]:
model_9 = models.Sequential()
model_9.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 1)))
model_9.add(layers.MaxPooling2D((2, 2)))
model_9.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_9.add(layers.MaxPooling2D((2, 2)))
model_9.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_9.add(layers.MaxPooling2D((2, 2)))
model_9.add(layers.Conv2D(256, (3, 3), activation='relu'))#brough another Convolution layer back
model_9.add(layers.MaxPooling2D((2, 2)))
model_9.add(layers.Flatten())
model_9.add(layers.Dense(64, activation='relu'))
model_9.add(layers.Dense(128, activation='relu'))
model_9.add(layers.Dense(256, activation='relu'))#brough another dense layer layer back
model_9.add(layers.Dense(5, activation='softmax'))

model_9.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

In [58]:
history_10 = model_9.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=10, 
    validation_data=validation_generator,
    validation_steps=50)

Epoch 1/10
100/100 [==============================] - 62s 623ms/step - loss: 1.6040 - acc: 0.2265 - val_loss: 1.6019 - val_acc: 0.2045
Epoch 2/10
100/100 [==============================] - 54s 536ms/step - loss: 1.5750 - acc: 0.2870 - val_loss: 1.5407 - val_acc: 0.3582
Epoch 3/10
100/100 [==============================] - 79s 792ms/step - loss: 1.5127 - acc: 0.3525 - val_loss: 1.5356 - val_acc: 0.3495
Epoch 4/10
100/100 [==============================] - 92s 918ms/step - loss: 1.4828 - acc: 0.3440 - val_loss: 1.5104 - val_acc: 0.3560
Epoch 5/10
100/100 [==============================] - 65s 651ms/step - loss: 1.4727 - acc: 0.3590 - val_loss: 1.5345 - val_acc: 0.3017
Epoch 6/10
100/100 [==============================] - 75s 747ms/step - loss: 1.4568 - acc: 0.3635 - val_loss: 1.5388 - val_acc: 0.3374
Epoch 7/10
100/100 [==============================] - 55s 554ms/step - loss: 1.4484 - acc: 0.3865 - val_loss: 1.6060 - val_acc: 0.3549
Epoch 8/10
100/100 [==============================] - 6

In [59]:
model_a = models.Sequential()
model_a.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 1)))
model_a.add(layers.MaxPooling2D((2, 2)))
model_a.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_a.add(layers.MaxPooling2D((2, 2)))
model_a.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_a.add(layers.MaxPooling2D((2, 2)))
model_a.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_a.add(layers.MaxPooling2D((2, 2)))
model_a.add(layers.Conv2D(256, (3, 3), activation='relu'))#brough another Convolution layer back
model_a.add(layers.MaxPooling2D((2, 2)))
model_a.add(layers.Flatten())
model_a.add(layers.Dense(64, activation='relu'))
model_a.add(layers.Dense(64, activation='relu'))
model_a.add(layers.Dense(128, activation='relu'))
model_a.add(layers.Dense(256, activation='relu'))#brough another dense layer layer back
model_a.add(layers.Dense(5, activation='softmax'))

model_a.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

In [60]:
history_a = model_a.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=17, 
    validation_data=validation_generator,
    validation_steps=50)

Epoch 1/17
100/100 [==============================] - 84s 840ms/step - loss: 1.6017 - acc: 0.2249 - val_loss: 1.6061 - val_acc: 0.1923
Epoch 2/17
100/100 [==============================] - 94s 942ms/step - loss: 1.5687 - acc: 0.2980 - val_loss: 1.5636 - val_acc: 0.2749
Epoch 3/17
100/100 [==============================] - 109s 1s/step - loss: 1.5206 - acc: 0.3305 - val_loss: 1.5276 - val_acc: 0.3286
Epoch 4/17
100/100 [==============================] - 68s 684ms/step - loss: 1.4965 - acc: 0.3465 - val_loss: 1.5554 - val_acc: 0.3275
Epoch 5/17
100/100 [==============================] - 65s 653ms/step - loss: 1.4844 - acc: 0.3565 - val_loss: 1.5631 - val_acc: 0.3363
Epoch 6/17
100/100 [==============================] - 62s 621ms/step - loss: 1.4666 - acc: 0.3625 - val_loss: 1.5143 - val_acc: 0.3855
Epoch 7/17
100/100 [==============================] - 79s 789ms/step - loss: 1.4720 - acc: 0.3625 - val_loss: 1.5167 - val_acc: 0.3890
Epoch 8/17
100/100 [==============================] - 84s

In [61]:
test_loss, test_acc = model_a.evaluate_generator(test_generator, steps=50)
print(test_loss)
print(test_acc)

1.41400719005152
0.3837471809196149
